In [2]:
import requests
import time
import pandas as pd
import smtplib
from datetime import datetime
from email.mime.text import MIMEText
import matplotlib.pyplot as plt

# Constants
API_KEY = '2bff56c00d1e2e022b04a3f354e94c26'  # Your API key
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather'
FORECAST_URL = 'http://api.openweathermap.org/data/2.5/forecast'
CITIES = ['Delhi', 'Mumbai', 'Chennai', 'Bangalore', 'Kolkata', 'Hyderabad']
ALERT_THRESHOLD_TEMP = 35  # Alert threshold for temperature in Celsius
UNITS = 'metric'  # Temperature in Celsius

# Initialize weather data storage
weather_data = []

# Function to get current weather data for a city
def get_weather(city):
    try:
        url = f"{BASE_URL}?q={city}&appid={API_KEY}&units={UNITS}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return {
                'city': city,
                'main': data['weather'][0]['main'],
                'temp': data['main']['temp'],
                'feels_like': data['main']['feels_like'],
                'humidity': data['main']['humidity'],
                'wind_speed': data['wind']['speed'],
                'dt': datetime.now()  # Use system time to capture real-time updates
            }
        else:
            print(f"Error retrieving data for {city}: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception occurred for {city}: {e}")
        return None

# Function to send email alert
def send_alert(city, temp):
    msg = MIMEText(f"Alert! {city} temperature exceeded {ALERT_THRESHOLD_TEMP}°C. Current temp: {temp}°C")
    msg['Subject'] = f"Weather Alert for {city}"
    msg['From'] = "pavithrapalni1282003@gmail.com"  # Your email
    msg['To'] = "pavithrapriya282003@gmail.com"  # Recipient email

    # SMTP configuration for Gmail
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()  # Start TLS encryption
        server.login("pavithrapalni1282003@gmail.com", "Pavithra@2003")  # Replace with your Gmail password
        server.send_message(msg)

    print(f"Email alert sent for {city}")

# Function to fetch weather data for all cities
def fetch_weather_data(interval=300):
    while True:
        for city in CITIES:
            data = get_weather(city)
            if data:
                weather_data.append(data)
                print(f"Weather data for {city}: {data}")

        # Process data every interval
        process_weather_data()
        time.sleep(interval)

# Function to calculate daily summary
def calculate_daily_summary(weather_data):
    df = pd.DataFrame(weather_data)
    df['date'] = df['dt'].dt.date  # Extract date

    # Daily rollups and aggregation
    daily_summary = df.groupby(['city', 'date']).agg({
        'temp': ['mean', 'max', 'min'],
        'humidity': 'mean',
        'wind_speed': 'mean',
        'main': lambda x: x.mode()[0]  # Dominant weather condition
    }).reset_index()

    return daily_summary

# Function to check alerts based on temperature
def check_alerts(weather_data):
    if not weather_data:  # If weather_data is empty
        print("No temperature data available for alert checking.")
        return

    # Iterate over the weather data to check for alerts
    for data in weather_data:
        if data['temp'] > ALERT_THRESHOLD_TEMP:  # Use your threshold
            send_alert(data['city'], data['temp'])  # Send an alert

# Function to process weather data
def process_weather_data():
    daily_summary = calculate_daily_summary(weather_data)
    print(daily_summary)
    
    # Check alerts
    check_alerts(weather_data)
    
    # Clear weather_data for the next interval
    weather_data.clear()

    # Visualize daily summaries
    visualize_daily_summary(daily_summary)

# Function to visualize daily summaries
def visualize_daily_summary(daily_summary):
    # Plotting average temperature for each city
    plt.figure(figsize=(10, 5))
    for city in daily_summary['city'].unique():
        city_data = daily_summary[daily_summary['city'] == city]
        plt.plot(city_data['date'], city_data[('temp', 'mean')], marker='o', label=city)

    plt.title('Daily Average Temperature by City')
    plt.xlabel('Date')
    plt.ylabel('Average Temperature (°C)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Function to get weather forecast data for a city
def get_forecast(city):
    url = f"{FORECAST_URL}?q={city}&appid={API_KEY}&units={UNITS}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        forecast_data = []
        for entry in data['list']:
            forecast_data.append({
                'city': city,
                'temp': entry['main']['temp'],
                'humidity': entry['main']['humidity'],
                'wind_speed': entry['wind']['speed'],
                'main': entry['weather'][0]['main'],
                'dt': datetime.fromtimestamp(entry['dt'])
            })
        return forecast_data
    else:
        print(f"Error retrieving forecast for {city}: {response.status_code}")
        return None

# Function to fetch forecasts for all cities
def fetch_forecast_data():
    forecast_data = []
    for city in CITIES:
        data = get_forecast(city)
        if data:
            forecast_data.extend(data)
    return forecast_data

# Example of fetching forecast data (optional to run)
forecast_data = fetch_forecast_data()
print(forecast_data)

# Start fetching real-time data
fetch_weather_data()


[{'city': 'Delhi', 'temp': 34.59, 'humidity': 24, 'wind_speed': 2.54, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 18, 17, 30)}, {'city': 'Delhi', 'temp': 32.01, 'humidity': 30, 'wind_speed': 2.68, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 18, 20, 30)}, {'city': 'Delhi', 'temp': 29.12, 'humidity': 36, 'wind_speed': 2.67, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 18, 23, 30)}, {'city': 'Delhi', 'temp': 27.67, 'humidity': 38, 'wind_speed': 1.97, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 19, 2, 30)}, {'city': 'Delhi', 'temp': 26.1, 'humidity': 43, 'wind_speed': 2.59, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 19, 5, 30)}, {'city': 'Delhi', 'temp': 27.51, 'humidity': 40, 'wind_speed': 2.84, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 19, 8, 30)}, {'city': 'Delhi', 'temp': 32.74, 'humidity': 26, 'wind_speed': 1.49, 'main': 'Clear', 'dt': datetime.datetime(2024, 10, 19, 11, 30)}, {'city': 'Delhi', 'temp': 35.72, 'humidity': 19, 'wind_speed': 0.

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-20e5a8f5c65sm8944695ad.205 - gsmtp')